## **Trabalho de IA**
William Alves da Silva


* **DATASET**
    O dataset do trabalho é uma lista com todos os Pokémons (721) da primeira geração, incluindo número, nome, primeiro e segundo tipo e estatísticas básicas: HP, Ataque, Defesa, Ataque Especial, Defesa Especial e Velocidade. 
    Esses são os atributos brutos que são usados para calcular quanto dano um ataque causará nos jogos. Este conjunto de dados é sobre os jogos de pokemon

* **OBJETIVOS**
    Objetivo deste notebook é apresentar os dados de diversas modos, para conehcer a informação que estamos trabalhando e também melhorar o conjunto de dados, para posteriormente aplicarmos Machine Learnning sobre o mesmo.
    Identificar as features (variaveis) que podemos utilizar para descobrir o tipo de um Pokemon. (Obs.: Ataque e Defesa não são suficientes para descobrir um tipo de pokemon)

### Importação das bibliotacas básicas/padrões para trabalharmos com manipulação de dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Input do [DATASET](https://www.kaggle.com/abcsds/pokemon)

In [ ]:
df =  pd.read_csv('../input/Pokemon.csv')

### Visualização prévia do conjunto de dados

In [ ]:
df.head(10)

### Algumas trativas e observações
* Removido underscore
* Colunas em maisculo para evitar digitação incorreta, etc.

In [ ]:
df.columns = df.columns.str.upper().str.replace('_', '') 
full_data = df
df.head()

### Quantidade de Pokemons por Tipo primario (TYPE 1)

In [ ]:
df['TYPE 1'].value_counts()

### Informações adicionais sobres os dados
* Aqui é possível observar que existem 800 registros, nenhuma da colunas é NULL, duas colunas são do tipo 'object', oito int64 e somente uma boleana.

In [ ]:
df.info()

### Informações estatisticas do DATASET
* Média, Desvio padrão, Minimo, Máximo, etc.

In [ ]:
df.describe()

### Alterado o index padrão para a coluna NAME

In [ ]:
df = df.set_index('NAME')
df.head()

### Observado que para do Pokemon Mega, é replicado o seu nome.
Exemplo: 
* Antigo: CharizardMega Charizard Y
* Novo: Mega Charizard Y

In [ ]:
df.index = df.index.str.replace(".*(?=Mega)", "")
df.head(10)


### Dropado a coluna #

In [ ]:
df=df.drop(['#'],axis=1)

### Alguns valores para o TYPE 2 estavam NaN (Not a Number)
Preenchido esses campos com o valor de TYPE 1

In [ ]:
df['TYPE 2'].fillna(df['TYPE 1'], inplace=True)

### Busca com operadores

In [ ]:
df[((df['TYPE 1']=='Fire') | (df['TYPE 1']=='Dragon')) & ((df['TYPE 2']=='Dragon') | (df['TYPE 2']=='Fire'))].head(3)

### Maior defesa com argmax e idmax

In [ ]:
print("MAx DEFENSE:",df['DEFENSE'].argmax())
print("MAx DEFENSE:",(df['DEFENSE']).idxmax())
print(df.iloc[224])
df.sort_values(by=['DEFENSE'], ascending=False).head(3)



## VISUALIZAÇÕES

### Distribuição de Ataque para todos os Pokemons

In [ ]:
bins=range(0,200,20) 
plt.hist(df["ATTACK"],bins,histtype="bar",rwidth=1.2,color='#0ff0ff') 

# Eixos
plt.xlabel('ATAQUE')
plt.ylabel('QTD')
plt.plot()

# Pontilhado para a média
plt.axvline(df['ATTACK'].mean(),linestyle='dashed',color='red') 
plt.show()

### Fire VS Water
A ideia é comparar todos os pokém de Fogo contra todos os pokemons de Água

In [ ]:
# Separação dos conjuntos
fire=df[(df['TYPE 1']=='Fire') | ((df['TYPE 2'])=="Fire")]
water=df[(df['TYPE 1']=='Water') | ((df['TYPE 2'])=="Water")]

plt.scatter(fire.ATTACK.head(50),fire.DEFENSE.head(50),color='R',label='Fire',marker="*",s=50)
plt.scatter(water.ATTACK.head(50),water.DEFENSE.head(50),color='B',label="Water",s=25)

# Eixos
plt.xlabel("ATAQUE")
plt.ylabel("DEFESA")

plt.legend()
plt.plot()
fig=plt.gcf()  

# Tamanho do quadro
fig.set_size_inches(20,10) 
plt.show()

Aqui é possível observar que os Pokemons de Fogo são possuem mas ataque em relação aos de Água, porém os de Água possuem mais defesa.

### Pokemons mais forte de cada tipo

In [ ]:
strong=df.sort_values(by='TOTAL', ascending=False) 
strong.drop_duplicates(subset=['TYPE 1'],keep='first')

## Análises estatísticas dos pokemons

In [ ]:
df2 = df.drop(['GENERATION','TOTAL'],axis=1)
sns.boxplot(data=df2)
plt.ylim(0,250) 
plt.show()

In [ ]:
plt.subplots(figsize = (20,5))
plt.title('Ataque por type 1')
sns.boxplot(x = "TYPE 1", y = "ATTACK",data = df)
plt.ylim(0,200)
plt.show()

In [ ]:
plt.subplots(figsize = (20,5))
plt.title('Ataque por type 2')
sns.boxplot(x = "TYPE 2", y = "ATTACK",data=df)
plt.show()

In [ ]:
plt.subplots(figsize = (15,5))
plt.title('Defesa por Type 1')
sns.boxplot(x = "TYPE 1", y = "DEFENSE",data = df)
plt.show()

### Pokemons mais forte por Tipo

In [ ]:
plt.figure(figsize=(20,10))
top_types=df['TYPE 1'].value_counts()
df1=df[df['TYPE 1'].isin(top_types.index)] 
sns.swarmplot(x='TYPE 1',y='TOTAL',data=df1,hue='LEGENDARY')
plt.axhline(df1['TOTAL'].mean(),color='red',linestyle='dashed')
plt.show()

### Encontranto qualquer correlação entre todos os atributos

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.corr(),annot=True)
plt.show()

A partir do mapa de calor, pode-se ver que não há muita correlação entre os atributos dos pokemons. O mais alto que podemos ver é a correlação entre Sp.Atk e Total

### Correlação dos atributos entre os Pokemons
A ideia aqui é tentar identificar alguns atributos que separem os pokemons lendários dos pokemons normais.

In [ ]:
datax = df.drop(["TYPE 1","TYPE 2","GENERATION"],axis=1)
_ =sns.pairplot(datax, hue='LEGENDARY', diag_kind='kde', height=2)
_ = 10

# Classificação
## Tentar classificar se um pokémon é lendário

### Separando (salvando) o conjunto de dados

In [ ]:
data = full_data.drop(['TYPE 2'],axis='columns')

In [ ]:
data.LEGENDARY.value_counts()

Devido a existir uma desiguldade entre lendários e normais, "igualado" esses valores
* Separado somente os lendários
* E multiplicado os mesmos

In [ ]:
lengendary = data.loc[data['LEGENDARY']==True]
lengendary = lengendary.append(lengendary.append(lengendary))

In [ ]:
lengendary.LEGENDARY.value_counts()

* Adicionado os lendários de volta ao conjunto (multiplicando de novo)

In [ ]:
full_data = data.append(lengendary.append(lengendary.append(lengendary)))

In [ ]:
full_data

### Mapeado os valores Boleanos

In [ ]:
full_data['LEGENDARY'] = full_data.LEGENDARY.map({False: 0, True: 1})

In [ ]:
full_data

### Bibliotecas para processamento de dados

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
xyz = make_column_transformer(
            (OneHotEncoder(),['TYPE 1','GENERATION']),
            (StandardScaler(),['TOTAL','HP','ATTACK','DEFENSE','SP. ATK','SP. DEF','SPEED']), remainder = 'passthrough')

### Treino e Teste
* Serando modelos de treino e modelos de teste

In [ ]:
full_data = full_data.drop(['#','NAME'],axis='columns')
X = full_data.drop(['LEGENDARY'], axis = 1)
y = full_data['LEGENDARY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
print(len(X), len(y), len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
xyz.fit_transform(X_train)

### Regressão logística

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
logreg = LogisticRegression(solver='lbfgs')
pipe = make_pipeline(xyz,logreg)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

print('Score do dataset de Treino: {}'.format(cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()*100))

pipe = make_pipeline(xyz,logreg)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print('Score do dataset de Teste: {}'.format(metrics.accuracy_score(y_test,y_pred)*100))

### Suporte Vector Machine 

In [ ]:
from sklearn.svm import SVC

svc_scores = []
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

for i in range(len(kernels)):
    svc_classifier = SVC(kernel = kernels[i])
    pipe = make_pipeline(xyz,svc_classifier)
    svc_scores.append(cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean())

In [ ]:
from matplotlib.cm import rainbow

colors = rainbow(np.linspace(0, 1, len(kernels)))
plt.figure(figsize=(10,7))
plt.bar(kernels, svc_scores, color = colors)

for i in range(len(kernels)):
    plt.text(i, svc_scores[i], svc_scores[i])
    
plt.xlabel('Kernels')
plt.ylabel('Scores')


In [ ]:
print('Score do dataset de Treino: {}'.format(svc_scores[0]*100))

svc_classifier = SVC(kernel = 'linear')
pipe = make_pipeline(xyz, svc_classifier)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print('Score do dataset de Teste:: {}'.format(metrics.accuracy_score(y_test,y_pred)*100))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_scores = []
estimators = [10, 100, 200, 500, 1000]

for i in estimators:
    rf_classifier = RandomForestClassifier(n_estimators = i, random_state = 0)
    pipe = make_pipeline(xyz, rf_classifier)
    rf_scores.append(cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean())

In [ ]:
plt.figure(figsize=(10,7))
colors = rainbow(np.linspace(0, 1, len(estimators)))
plt.bar([i for i in range(len(estimators))], rf_scores, color = colors, width = 0.8)

for i in range(len(estimators)):
    plt.text(i, rf_scores[i], round(rf_scores[i],5))
plt.xticks(ticks = [i for i in range(len(estimators))], labels = [str(estimator) for estimator in estimators])

plt.xlabel('Estimadores')
plt.ylabel('Score')


In [ ]:
print('Score do dataset de Treino: {}'.format(rf_scores[0]*100))

rf_classifier = RandomForestClassifier(n_estimators = 10, random_state = 0)
pipe = make_pipeline(xyz,rf_classifier)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print('Score do dataset de Teste: {}'.format(metrics.accuracy_score(y_test,y_pred)*100))